## Imports

In [1]:
import numpy as np
import pickle
import os
from MyDataset import MyDataset
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from torchvision import transforms, models
from torch.nn import functional as F
from torch.autograd import Variable
from torch.optim import lr_scheduler
from tqdm.auto import tqdm
from PIL import Image
%matplotlib inline

## Transform raw data

In [2]:
pre_transform = transforms.Compose([
    #transforms.RandomHorizontalFlip(),
    #transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5),
    #transforms.RandomCrop(200),
    #transforms.RandomVerticalFlip(),
    transforms.Resize((224,224)),
    transforms.ToTensor()
])
val_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor()
])

## Load Data

In [3]:
train_dataset = MyDataset("./train",pre_transform)
val_dataset = MyDataset("./validate",val_transform)

In [4]:
print("train data length: %d  valid data length % d"%(len(train_dataset),len(val_dataset)))

train data length: 1754  valid data length  306


## Visualize Traning Data

In [5]:
# img,label = train_dataset[0]
# if (label == 0):
#     print("This is class nil")
# elif (label == 1):
#     print("This is class MOD")
# else:
#     print("This is class SEV")

# Band8 = img[0][:][:]
# Band12 = img[1][:][:]
# Band13 = img[2][:][:]
# Band14 = img[3][:][:]
# Band = [Band8,Band12,Band13,Band14]

# for i in range(1,5):
#     plt.subplot(2,2,i)
#     plt.imshow(Band[i-1])
# plt.show()

In [6]:
# #Calculate the number of samples in differnet class
# train_nil_count = 0   ##591
# train_MOD_count = 0   ##839
# train_SEV_count = 0   ##324
# valid_nil_count = 0   ##192
# valid_MOD_count = 0   ##81
# valid_SEV_count = 0   ##33

# for i in range (0,1754):
#     #print(i)
#     img, label = train_dataset[i]
#     if (label == 0):
#         train_nil_count += 1
#     elif (label == 1):
#         train_MOD_count += 1
#     else:
#         train_SEV_count += 1
# print("train set has %d nil, %d MOD, %d SEV" %(train_nil_count, train_MOD_count, train_SEV_count))

# for i in range (0,306):
#     #print(i)
#     img, label = val_dataset[i]
#     if (label == 0):
#         valid_nil_count += 1
#     elif (label == 1):
#         valid_MOD_count += 1
#     else:
#         valid_SEV_count += 1
# print("valid set has %d nil, %d MOD, %d SEV" %(valid_nil_count, valid_MOD_count, valid_SEV_count))

## Define Model & Loss

In [7]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(4, 16, 3, 1, 1)
        self.conv2 = nn.Conv2d(16, 32, 3, 1, 1)
        self.conv3 = nn.Conv2d(32, 64, 3, 1, 1)
        self.fc1 = nn.Linear(64*28*28, 2048)
        self.fc2 = nn.Linear(2048, 100)
        self.fc3 = nn.Linear(100, 3)
        self.batchnorm1 = nn.BatchNorm2d(16)
        self.batchnorm2 = nn.BatchNorm2d(32)
        self.batchnorm23 = nn.BatchNorm2d(64)
        self.dropout = nn.Dropout(0.1)
        self.batchnorm3 = nn.BatchNorm1d(2048)
        self.batchnorm4 = nn.BatchNorm1d(100)
        self.batchnorm5 = nn.BatchNorm1d(3)
        
    def forward(self, x):
        # x's shape = N * 4 * 224 * 224, where N is batch size
        x = self.conv1(x)
        # x's shape = N * 16 * 224 * 224
        x = self.batchnorm1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        # x's shape = N * 16 * 112 * 112
        x = self.conv2(x)
        # x's shape = N * 32 * 112 * 112
        x = self.batchnorm2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        # x's shape = N * 32 * 56 * 56
        x = self.conv3(x)
        x = self.batchnorm23(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = torch.flatten(x, 1) # flat starts from the second dim
        # x's shape = N * (32 * 56 * 56) = N * 100352
        x = self.fc1(x)
        # x's shape =  N * 128
        x = self.batchnorm3(x)
        x = F.relu(x)
        # x =  self.dropout(x)
        # x's shape =  N * 128
        # x = self.dropout(x)
        x = self.fc2(x)
        x = self.batchnorm4(x)
        x = F.relu(x)
        # x's shape =  N * 3
        # x =  self.dropout(x)
        x = self.fc3(x)
        x = self.batchnorm5(x)
        # x = F.relu(x)
        # x's shape =  N * 3
        # x =  self.dropout(x)
        return x

In [8]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True, pin_memory=False)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=64, shuffle=True, pin_memory=False)

In [9]:
model = Net()
print(model)

Net(
  (conv1): Conv2d(4, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=50176, out_features=2048, bias=True)
  (fc2): Linear(in_features=2048, out_features=100, bias=True)
  (fc3): Linear(in_features=100, out_features=3, bias=True)
  (batchnorm1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm23): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (batchnorm3): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm4): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm5): BatchNorm1d(3, eps=1e-05, momentum

In [10]:
criterion = nn.CrossEntropyLoss()

## LDAM Loss

In [11]:
class LDAMLoss(nn.Module):
    
    def __init__(self, cls_num_list, max_m=0.5, weight=None, s=30):
        super(LDAMLoss, self).__init__()
        m_list = 1.0 / np.sqrt(np.sqrt(cls_num_list))
        m_list = m_list * (max_m / np.max(m_list))
        m_list = torch.cuda.FloatTensor(m_list)
        self.m_list = m_list
        assert s > 0
        self.s = s
        self.weight = weight

    def forward(self, x, target):
        index = torch.zeros_like(x, dtype=torch.uint8)
        index.scatter_(1, target.data.view(-1, 1), 1)
        
        index_float = index.type(torch.cuda.FloatTensor)
        batch_m = torch.matmul(self.m_list[None, :], index_float.transpose(0,1))
        batch_m = batch_m.view((-1, 1))
        x_m = x - batch_m
    
        output = torch.where(index, x_m, x)
        return F.cross_entropy(self.s*output, target, weight=self.weight)

In [12]:
def LDAMLoss(output, target, n_class_nil, n_class_MOD, n_class_SEV, C):
    loss = 0
    n_class = [n_class_nil, n_class_MOD, n_class_SEV]
    #output_class = output.argmax(dim=1, keepdim=False) #Shape [64], the predict class
    niter = output.shape[0] # 64 for a batch
    for i in range(niter):
        Z_y = output[i][target[i]]
        delta_y = C / (n_class[target[i]] ** (0.25) )
        nominator = torch.exp(Z_y - delta_y)
        if (target[i] == 0):
            dinominator = nominator + torch.exp(output[i][1]) + torch.exp(output[i][2])
        elif (target[i] == 1):
            dinominator = nominator + torch.exp(output[i][0]) + torch.exp(output[i][2])
        else:
            dinominator = nominator + torch.exp(output[i][0]) + torch.exp(output[i][1])
        loss += (-torch.log(nominator/dinominator))
    return loss

In [13]:
def compute_acc(outputs, targets):
    preds = outputs.argmax(dim=1, keepdim=True)
    return preds.eq(targets.view_as(preds)).sum().item() / targets.shape[0]

In [14]:
def performance_eval(outputs, targets):
    preds = outputs.argmax(dim=1, keepdim=False)
    targets = targets.view_as(preds)
    performance_dict = {"NIL":0.0,"MOD":0.0, "SEV":0.0 }
    for i in range(3):
        TP = targets[preds.eq(i)].eq(i).sum().item()
        FP = (~targets[preds.eq(i)].eq(i)).sum().item()
        FN = (~preds[targets.eq(i)].eq(i)).sum().item()
        Precision =  torch.true_divide(TP,(TP + FP))
        Recall = torch.true_divide(TP,(TP + FN))
        F1 = torch.true_divide(2*(Precision*Recall),(Precision+Recall))
        performance_dict[list(performance_dict.keys())[i]] = torch.tensor([Precision,Recall,F1])
    return performance_dict

In [15]:
class AverageMeter(object):
    def __init__(self):
        self.reset()
        
    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.cnt = 0
        self.TP = torch.tensor([0,0,0])
        self.FP = torch.tensor([0,0,0])
        self.FN = torch.tensor([0,0,0])
        
    def update(self, val, outputs, targets, n=1):
        self.val = val
        self.sum += val * n
        self.cnt += n
        self.avg = self.sum / self.cnt
        preds = outputs.argmax(dim=1, keepdim=True)
        preds = preds.view(-1)
        targets = targets.view(-1)
        for i in range(3):
            self.TP[i] += targets[preds.eq(i)].eq(i).sum().item()
            self.FP[i] += (~targets[preds.eq(i)].eq(i)).sum().item()
            self.FN[i] += (~preds[targets.eq(i)].eq(i)).sum().item()

In [16]:
model = model.cuda()
#criterion = criterion.cuda()

In [17]:
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [18]:
def train_one_epoch(inputs, targets, optimizer):
    # accs = AverageMeter()
    # for X, y in tqdm(train_loader, leave=False):
    # inputs = X.cuda()
    # targets = y.cuda()
    #inputs = X
    #targets = y
    optimizer.zero_grad()
    # forward
    outputs = model(inputs)
    loss =  LDAMLoss(outputs,targets,591,839,324,C = 1.0)
    loss.backward()
    optimizer.step()
    # accs.update(compute_acc(outputs, targets), outputs, targets, X.size(0))
        
    return loss, outputs 

In [19]:
def validate_one_epoch(inputs, targets):
    accs = AverageMeter()
    # for X, y in tqdm(val_loader, leave=False):
    # inputs = X.cuda()
    # targets = y.cuda()
    #inputs = X
    #targets = y
    #optimizer.zero_grad()
    # forward
    outputs = model(inputs)
    #loss = criterion(outputs, targets)
    loss = LDAMLoss(outputs,targets,192,81,33,C = 1)

    #loss.backward()
    #optimizer.step()
    accs.update(compute_acc(outputs, targets), outputs, targets, X.size(0))
    return loss, outputs  

In [20]:
best_epoch = -1
best_acc = 0.0
best_model_state = model.state_dict()
history_train_acc = []
history_val_acc = []

scheduler = lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

for epoch in tqdm(range(50)):
    for phase in range (0,2):
        if phase == 0:
            model.train()
            accs = AverageMeter()
            for X, y in tqdm(train_loader, leave=False):
                inputs = Variable(X.cuda())
                targets = Variable(y.cuda())
                loss, outputs = train_one_epoch(inputs, targets, optimizer)
                accs.update(compute_acc(outputs, targets), outputs, targets, X.size(0))
            print("epoch {} train acc: {:.4f} ".format(epoch, accs.avg))
            class_names = ["NIL","MOD","SEV"]
            scheduler.step()
            for i in range(3):
                Precision =  torch.true_divide(accs.TP[i],(accs.TP[i] + accs.FP[i]))
                Recall = torch.true_divide(accs.TP[i],(accs.TP[i] + accs.FN[i]))
                F1 = torch.true_divide(2*(Precision*Recall),(Precision+Recall))
                print("class {} Precision {:.3f} Recall{:.3f} F1 {:.3f}".format(class_names[i],Precision,Recall,F1))
            history_train_acc.append(accs)
            
        elif phase == 1:
            model.eval()
            accs = AverageMeter()
            for X, y in tqdm(val_loader, leave=False):
                inputs = Variable(X.cuda())
                targets = Variable(y.cuda())
                loss, outputs = validate_one_epoch(inputs, targets)
                accs.update(compute_acc(outputs, targets), outputs, targets, X.size(0))
            print("epoch {} valid acc: {:.4f} ".format(epoch, accs.avg))
            class_names = ["NIL","MOD","SEV"]
            for i in range(3):
                Precision =  torch.true_divide(accs.TP[i],(accs.TP[i] + accs.FP[i]))
                Recall = torch.true_divide(accs.TP[i],(accs.TP[i] + accs.FN[i]))
                F1 = torch.true_divide(2*(Precision*Recall),(Precision+Recall))
                print("class {} Precision {:.3f} Recall{:.3f} F1 {:.3f}".format(class_names[i],Precision,Recall,F1))
            history_val_acc.append(accs)
            
            if accs.avg > best_acc:
                best_acc = accs.avg
                best_epoch = epoch
                best_model_state = model.state_dict().copy()
            
print(f'[Info] best val acc: {best_acc:.2%} at {best_epoch+1}th epoch')

epoch 0 train acc: 0.5405 
class NIL Precision 0.710 Recall0.526 F1 0.604
class MOD Precision 0.624 Recall0.553 F1 0.586
class SEV Precision 0.302 Recall0.534 F1 0.386


epoch 0 valid acc: 0.6275 
class NIL Precision 0.627 Recall1.000 F1 0.771
class MOD Precision nan Recall0.000 F1 nan
class SEV Precision nan Recall0.000 F1 nan


RuntimeError: CUDA out of memory. Tried to allocate 392.00 MiB (GPU 0; 6.00 GiB total capacity; 3.88 GiB already allocated; 30.63 MiB free; 4.13 GiB reserved in total by PyTorch)
Exception raised from malloc at ..\c10\cuda\CUDACachingAllocator.cpp:272 (most recent call first):
00007FFC2B0E75A200007FFC2B0E7540 c10.dll!c10::Error::Error [<unknown file> @ <unknown line number>]
00007FFC2B089C0600007FFC2B089B90 c10_cuda.dll!c10::CUDAOutOfMemoryError::CUDAOutOfMemoryError [<unknown file> @ <unknown line number>]
00007FFC2B09069600007FFC2B08F370 c10_cuda.dll!c10::cuda::CUDACachingAllocator::init [<unknown file> @ <unknown line number>]
00007FFC2B09083A00007FFC2B08F370 c10_cuda.dll!c10::cuda::CUDACachingAllocator::init [<unknown file> @ <unknown line number>]
00007FFC2B08509900007FFC2B084EB0 c10_cuda.dll!c10::cuda::CUDAStream::unpack [<unknown file> @ <unknown line number>]
00007FFBE9F51FF100007FFBE9F51EB0 torch_cuda.dll!at::native::empty_cuda [<unknown file> @ <unknown line number>]
00007FFB

In [ ]:
train_accuracy = []
val_accuracy = []
for accs in history_train_acc:
    train_accuracy.append(accs.avg)
for accs in history_val_acc:
    val_accuracy.append(accs.avg)
plt.plot(range(50),train_accuracy)
plt.plot(range(50),val_accuracy)
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train','validation'], loc='upper left')
plt.show()

In [ ]:
torch.save(best_model_state, "3CNN3FC.pt")

In [ ]:
model.load_state_dict(torch.load("Baseline1&2_state.tar"))

In [ ]:
model.eval()
for X, y in tqdm(val_loader, leave=False):
    inputs = X.cuda()
    outputs = model(inputs)
    print("y actual:",y)
    print(outputs.argmax(dim=1, keepdim=True).view(-1))
    acc = compute_acc(outputs.cpu(), y)
    print(f'Test Acc: {acc}')

In [ ]:
final_evaluation()

In [ ]:
def final_evaluation():
    model.eval()
    accs = AverageMeter()
    for X, y in tqdm(train_loader, leave=False):
        inputs = X.cuda()
        targets = y.cuda()
        outputs = model(inputs)
        accs.update(compute_acc(outputs, targets), outputs, targets, X.size(0))
    print("train acc: {:.4f} ".format(accs.avg))
    class_names = ["NIL","MOD","SEV"]
    for i in range(3):
        Precision =  torch.true_divide(accs.TP[i],(accs.TP[i] + accs.FP[i]))
        Recall = torch.true_divide(accs.TP[i],(accs.TP[i] + accs.FN[i]))
        F1 = torch.true_divide(2*(Precision*Recall),(Precision+Recall))
        print("class {} Precision {:.3f} Recall{:.3f} F1 {:.3f}".format(class_names[i],Precision,Recall,F1))
        
    accs = AverageMeter()
    for X, y in tqdm(val_loader, leave=False):
        inputs = X.cuda()
        targets = y.cuda()
        outputs = model(inputs)
        accs.update(compute_acc(outputs, targets), outputs, targets, X.size(0))
    print("validation acc: {:.4f} ".format(accs.avg))
    class_names = ["NIL","MOD","SEV"]
    for i in range(3):
        Precision =  torch.true_divide(accs.TP[i],(accs.TP[i] + accs.FP[i]))
        Recall = torch.true_divide(accs.TP[i],(accs.TP[i] + accs.FN[i]))
        F1 = torch.true_divide(2*(Precision*Recall),(Precision+Recall))
        print("class {} Precision {:.3f} Recall{:.3f} F1 {:.3f}".format(class_names[i],Precision,Recall,F1))

In [ ]:
img, label = train_dataset.__getitem__(1700)

In [ ]:
img

In [ ]:
with open("train/severe_HS_H08_20180124_0400_33.98652512_104.272.npy.pickle", 'rb') as f:
    color_img = pickle.load(f)

In [ ]:
img = torch.tensor(color_img)
img = img.permute(1,2,0)
img = img/255
img.shape

In [ ]:
plt.imshow(img.permute(1,2,0))